# Train a JointVAE model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import torch

use_cuda = torch.cuda.is_available()
use_cuda

True

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # change to your device

#### Prepare data list

#### Create list of image paths

In [4]:
#loadable_dresses = list(np.loadtxt('data/loadable_women_primary_dress.csv',delimiter=',',skiprows=1,dtype='str'))

In [5]:
#print(len(loadable_dresses))

In [6]:
#loadable_dresses[:5]

In [7]:
!ls /workspace/mnt/crucial_2TB/111_Extra_Data/Macys/

all_wearables20180810  imgs224_1000  xtrain3150_260260.csv
archive28.tar.bz2      imgs260_3150  xtrainonly1000.csv


In [22]:
dir_name = '/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/all_wearables20180810/2/8/'
files = [dir_name + x for x in os.listdir(dir_name) if '.jpg' in x]
files[:3]

['/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/all_wearables20180810/2/8/2847897_3623373.jpg',
 '/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/all_wearables20180810/2/8/2866265_3648851.jpg',
 '/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/all_wearables20180810/2/8/2880939_3410936.jpg']

In [23]:
len(files)

122157

#### Split into train and test set

In [29]:
#image_paths_train = loadable_dresses[:88800]
#image_paths_test = loadable_dresses[88800:-124]
image_paths_train = files[:98000]
image_paths_test = files[98000:-57]


print(f"Number of train image paths: {len(image_paths_train):,d}")
print(f"Number of test image paths: {len(image_paths_test):,d}")
print()
print("Sample paths:")
print(image_paths_train[0])
print(image_paths_train[-1])
print(image_paths_test[0])
print(image_paths_test[-1])

Number of train image paths: 98,000
Number of test image paths: 24,100

Sample paths:
/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/all_wearables20180810/2/8/2847897_3623373.jpg
/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/all_wearables20180810/2/8/2807992_3592575.jpg
/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/all_wearables20180810/2/8/2856148_3749603.jpg
/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/all_wearables20180810/2/8/2890448_3718224.jpg


In [52]:
from torchvision import transforms
#from utils.dataloaders_custom import get_imagelist_dataloader, ImageListDataset
from utils.dataloader_tools import get_imagelist_dataloader, ImageListDataset

BATCH_SIZE = 100

composed = transforms.Compose([transforms.CenterCrop((256,256)),transforms.Resize((256,256)),transforms.ToTensor()])

# convert rgb is for the cv2 loaded images that I've got in this dir
conv_rgb = False
error_handling = True
train_dataset = ImageListDataset(image_paths_train, cut_from='top', cut_amount=256, 
                                 transform=composed, convert_rgb=conv_rgb, error_handling=True)
test_dataset = ImageListDataset(image_paths_test, cut_from='top', cut_amount=256, 
                                 transform=composed, convert_rgb=conv_rgb, error_handling=True)

train_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=train_dataset)
test_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=test_dataset)

### Define latent distribution of the model

In [53]:
# Latent distribution will be joint distribution of 10 gaussian normal distributions
# and one 10 dimensional Gumbel Softmax distribution
latent_spec = {'cont': 10,
               'disc': [10]}

### Build a model

In [54]:
!ls jointvae

__init__.py	       models_128_xstyle_int_debug.py
__pycache__	       models_128_xstyle_int_nd.py
decoder_scratchpad.py  models_256_xstyle_int_nd.py
encoder_scratchpad.py  models_64_xstyle.py
models.py	       models_64_xstyle_finished_v1.py
models_128_v1.py       models_64_xstyle_int_debug.py
models_128_v2.py       models_64_xstyle_int_nd.py
models_128_v3.py       training.py
models_128_v4.py       training_debug.py


In [55]:
from jointvae.models_256_xstyle_int_nd import VAE

#model = VAE(latent_spec=latent_spec, img_size=(3, 260, 260), use_cuda=use_cuda)
model = VAE(latent_spec=latent_spec, img_size=(3, 256, 256), use_cuda=use_cuda)

In [56]:
#print(model)

### Train the model

In [57]:
from torch import optim

# Build optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4, amsgrad=True) # added amsgrad # orig lr 5e-4

In [58]:
from jointvae.training import Trainer
#from jointvae.training_debug import Trainer

# Define the capacities
# Continuous channels
cont_capacity = [0.0, 4.8, 30000, 32.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0
# Discrete channels
disc_capacity = [0.0, 4.8, 30000, 32.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0

# Build a trainer
trainer = Trainer(model, optimizer,
                  cont_capacity=cont_capacity,
                  disc_capacity=disc_capacity,
                 use_cuda=use_cuda)

#### Initialize visualizer

In [59]:
#from viz.visualize import Visualizer
from viz.visualize import Visualizer

viz = Visualizer(model)
viz.save_images = False # needed to add this so it returns a tensor

In [ ]:
# Train model for 10 epochs
# Note this should really be a 100 epochs and trained on a GPU, but this is just to demo

trainer.train(train_loader, epochs=600, save_training_gif=None)

0/98000	Loss: 45938.223


In [ ]:
print('hi')

In [ ]:
# Get a batch of data
for batch, labels in test_loader:
    break
print("batch: ",type(batch),batch.shape)
type(viz.reconstructions(batch))

### Visualize

In [ ]:
# Plot reconstructions
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Get a batch of data
for batch, labels in test_loader:
    break

# Reconstruct data using Joint-VAE model
recon = viz.reconstructions(batch)

plt.figure(figsize=(26,26))
plt.imshow(np.rot90(np.transpose(recon.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e600_b100_c10d10_cap48_30k_gam32_reconstructions.png",dpi=200)

In [ ]:
# Plot samples
samples = viz.samples()

plt.figure(figsize=(26,26))
plt.imshow(np.rot90(np.transpose(samples.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e600_b100_c10d10_cap48_30k_gam32_samples.png",dpi=200)

#### Traverses all latent dimensions one by one and plots a grid of images where each row corresponds to a latent traversal of one latent dimension

In [ ]:
# Plot all traversals
traversals = viz.all_latent_traversals(size=10)

plt.figure(figsize=(20,20))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e600_b100_c10d10_cap48_30k_gam32_all_traversals_n20.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=2, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(20,20))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e600_b100_c10d10_cap48_30k_gam32_traversals2100.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=1, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(20,20))
plt.imshow(traversals.numpy()[0, :, :]);
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e600_b100_c10d10_cap48_30k_gam32_traversals1100.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=9, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(20,20))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e600_b100_c10d10_cap48_30k_gam32_traversals9100.png",dpi=200)

In [ ]:
!ls

### Save Model

In [ ]:
model_name = "xint256_e600_b100_c10d10_cap48_30k_gam32.pth"

In [ ]:
torch.save(model.state_dict(),"trained_models" + "statedict_" + model_name) # save state dict
#torch.save(model, model_name) # save full model

In [ ]:
print("Done training: ",model_name)

#### Restore Model from State Dict

In [ ]:
sd_model = VAE(latent_spec=latent_spec, img_size=(3, 64, 64))
sd_model.load_state_dict(torch.load("statedict_" + model_name))

#### Restore Full Model
* Note in this case the serialized data is bound to the specific classes and exact directory strucutre used.

In [ ]:
full_model = torch.load(model_name)

In [ ]:
type(full_model)

In [ ]:
type(sd_model)

In [ ]:
!ls